Instalação das packages e importação das mesmas

In [ ]:
%pip install transformers datasets torch optuna sentencepiece 
import torch
import optuna
from transformers import T5ForConditionalGeneration, T5Tokenizer, T5Config, Trainer, TrainingArguments, pipeline, EarlyStoppingCallback
from datasets import load_dataset

Load do dataset "_multi news_" da bilbioteca _datasets_ que contem notícias e o respetivo sumário
<br>O dataset vem com 3 splits, _train, validation e test_
<br>Para o treino e selecionou-se só parte do dataset devido a este ser muito grande e necessitar de demasiada memória no treino do modelo


In [ ]:
dataset = load_dataset("multi_news") #load ao dataset

train_size = int(0.05 * len(dataset["train"])) #arranjar x% do tamanho do dataset

reduced_train_dataset = dataset["train"].select(range(train_size)) #selecionar só x% do dataset de treino
val_dataset = dataset["validation"] #seleccionar o dataset de validação

Função de tokenização usando o modelo  _T5Tokenizer_ e ficheiro de vocabulário _t5-base_
<br> Retorna _input ids, attention mask e labels_ que são usados no treino do modelo
<br> Feita a tokenização do dataset de treino e de validação

In [2]:
def tokenize(batch):
    tokenizer = T5Tokenizer.from_pretrained("t5-base") # carrega o tokenizer T5 com o vocabulário pré-treinado do t5-base
    #aqui o max length equivale ao numero de tokens que são aceites em ambos os tokenizers
    tokenized_input = tokenizer(batch['document'], padding=True, truncation=True, return_tensors="pt", max_length=512) # faz tokenização das notícias, o que será a "entrada" na nossa aplicação
    tokenized_target = tokenizer(batch['summary'], padding=True, truncation=True, return_tensors="pt", max_length=150) # faz tokenização dos sumários, ou "saída"
    return {'input_ids': tokenized_input['input_ids'], 'attention_mask': tokenized_input['attention_mask'], 
            'labels': tokenized_target['input_ids']} # validar // t5 necessita de input ids, attention mask e labels para treinar

tokenized_train_dataset = reduced_train_dataset.map(tokenize, batched=True, batch_size=16) # aqui o map vai aplicar a tokenização a todos os elementos do dataset
tokenized_val_dataset = val_dataset.map(tokenize, batched=True, batch_size=16) # aqui o map vai aplicar a tokenização a todos os elementos do dataset



Found cached dataset multi_news (/home/azureuser/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/azureuser/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72/cache-871d40ca3f50d3b5.arrow
Loading cached processed dataset at /home/azureuser/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72/cache-7506a25ba01408fb.arrow


Carrega a arquitetura da rede neuronal T5 a ser usada para treinar o nosso modelo e o tokenizer com o vocabulário "t5-base"

In [3]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")

/anaconda/envs/jupyter_env/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Aqui é definido os parametros de treino do modelo, é criado o modelo e é treinado o modelo com variação de hiperparametros

In [4]:
def model_training(trial):
    # Teste hyierparametros, 3 valores diferentes para learning rate, weight decay e epochs, usados com combinações aleatórias 
    learning_rate = trial.suggest_categorical(
        "learning_rate", [1e-5, 2e-5 ,5e-5, 1e-4])
    weight_decay = trial.suggest_categorical("weight_decay", [0.0, 0.01, 0.005, 0.001])
    num_epochs = trial.suggest_categorical("num_epochs", [1,2,3])
       
    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=num_epochs, #numero de vezes que o dataset vai ser corrido no treino
        per_device_train_batch_size=4, # batch size refere-se à quantidade de samples que são processadas de uma vez pelo gpu neste caso
        per_device_eval_batch_size=4, # batch size refere-se à quantidade de samples que são processadas de uma vez pelo gpu neste caso
        gradient_accumulation_steps=4, # usado p reduzir batches em mini batches quando fica muito pesado em termos de memória
        evaluation_strategy="epoch", #tipo de avaliação do modelo, estamos a utilizar avaliação por epoch
        fp16=True, # mixed precision, torna o modelo mais eficiente ao utilizar valores float de 16 e 32 bits, o que pode aumentar velocidade e torna mais eficiente em memória, só funciona se for utilizado CUDA
        save_strategy="epoch",
        save_total_limit=10, #nm de modelos que grava
        logging_dir="./logs",
        learning_rate=learning_rate, #pensar no declive, quanto maior pode saltar dados e n converge, quanto menor pode demorar mt a convergir e pode não atingir min globais
        weight_decay=weight_decay,
        load_best_model_at_end=True,
        metric_for_best_model="loss",
        greater_is_better=False,
    )


    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_val_dataset,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)], #define quantos epochs precisa sem melhoria para parar de treinar 
    )

    trainer.train()
    metrics = trainer.evaluate()
    return metrics["eval_loss"]


In [5]:
aval = optuna.create_study(direction="minimize") #otimizar hiperparametros para minimizar loss do modelo
aval.optimize(model_training, n_trials=10) #quantas combinações de hiperparametros vai fazer, se for 10 ele vai fazer 10 combinações possíveis

[I 2023-04-13 17:04:55,006] A new study created in memory with name: no-name-f821208c-2185-407d-9c59-c90bef7fb7f9
/anaconda/envs/jupyter_env/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,No log,3.188620


Attempted to log scalar metric eval_loss:
2.535486936569214
Attempted to log scalar metric eval_runtime:
61.6075
Attempted to log scalar metric eval_samples_per_second:
4.561
Attempted to log scalar metric eval_steps_per_second:
1.152
Attempted to log scalar metric epoch:
1.0
Attempted to log scalar metric eval_loss:
2.5067856311798096
Attempted to log scalar metric eval_runtime:
62.6843
Attempted to log scalar metric eval_samples_per_second:
4.483
Attempted to log scalar metric eval_steps_per_second:
1.133
Attempted to log scalar metric epoch:
1.0
Attempted to log scalar metric eval_loss:
3.18861985206604
Attempted to log scalar metric eval_runtime:
62.6416
Attempted to log scalar metric eval_samples_per_second:
4.486
Attempted to log scalar metric eval_steps_per_second:
1.133
Attempted to log scalar metric epoch:
1.0
Attempted to log scalar metric train_runtime:
1536.1302
Attempted to log scalar metric train_samples_per_second:
1.463
Attempted to log scalar metric train_steps_per_sec

Attempted to log scalar metric eval_loss:
3.18861985206604
Attempted to log scalar metric eval_runtime:
61.6706
Attempted to log scalar metric eval_samples_per_second:
4.556
Attempted to log scalar metric eval_steps_per_second:
1.151
Attempted to log scalar metric epoch:
1.0


Epoch,Training Loss,Validation Loss
0,No log,2.806576


Attempted to log scalar metric eval_loss:
2.8065762519836426
Attempted to log scalar metric eval_runtime:
62.6641
Attempted to log scalar metric eval_samples_per_second:
4.484
Attempted to log scalar metric eval_steps_per_second:
1.133
Attempted to log scalar metric epoch:
1.0


In [8]:
best_params = aval.best_params #melhores hiperparametros de todos os testes feitos
print("Best hyperparameters:", best_params)

Best hyperparameters: {'learning_rate': 1e-05, 'weight_decay': 0.01, 'num_epochs': 1}


In [ ]:
def summarize(text):
    summarizer = pipeline("summarization", model="./results/checkpoint-843", tokenizer="t5-base")

    t5_input = "summarize: " + text # o modelo t5 precisa de ter "summarize: " como prefixo p sumarizar

    summary = summarizer(t5_input, min_length=30, max_length=150, num_beams=4, early_stopping=True)
    summarized_text = summary[0]["summary_text"]

    print(summarized_text)
